# 2.2.3 Advanced Connection Management

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#What-Youll-Learn">What You'll Learn</a></li>
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Architecture-Overview">Architecture Overview</a></li>
<li><a href="#1-Create-RDS-Proxy-and-IAM-Resources">1. Create RDS Proxy and IAM Resources</a></li>
<li><a href="#2-Create-Database-User-and-Connect">2. Create Database User and Connect</a></li>
<li><a href="#3-Monitoring-and-Optimization">3. Monitoring and Optimization</a></li>
<li><a href="#Next-Steps">Next Steps</a></li>
</ul>
</div>

Welcome to the final part of connecting to Aurora PostgreSQL! In this notebook, we'll explore advanced connection management using AWS services. 🚀

## What You'll Learn
- Connect using IAM authentication
- Use AWS Secrets Manager for credential management
- Leverage RDS Proxy for connection pooling
- Monitor and optimize database connections

## Prerequisites
- Completed [2.2.2 Working with Data](2.2.2_Working_with_Data.ipynb)
- Aurora cluster with IAM authentication, Secrets Manager, and RDS Proxy already set up in Part 2

## Cost Overview 💰

Use [AWS Pricing Calculator](https://calculator.aws/#/) to estimate the cost for your architecture solution. The following a rough cost estimation for the resources created in this notebook.

| Component | Cost (us-east-1) | Notes |
|-----------|------------------|--------|
| RDS Proxy | \$0.015/proxy-hour | Fully managed, highly available database proxy for Amazon RDS |
| Secrets Manager | \$0.40/secret-month | App user credentials storage |
| CloudWatch Alarms | \$0.10/alarm-month | 3 alarms for monitoring |
| IAM Roles/Policies | Free | No additional charges |

💡 **Cost Optimization Tips:**
- RDS Proxy provides connection pooling benefits that can potentially reduce Aurora ACU usage due to idle connections
- Secrets Manager includes automatic rotation capabilities
- CloudWatch alarms help prevent cost overruns through proactive monitoring
- Consider using fewer alarms initially and add more as needed

> **Free Tier Benefits**: New AWS accounts get 30 secrets free for 30 days in Secrets Manager, and 10 CloudWatch alarms free per month.Check [here for up-to-date Free Tier information](https://aws.amazon.com/free/).

## Architecture Overview
![Advanced Connection Architecture](../images/2.1-architecture-connect-to-db-using-rds-proxy.png)

This approach provides:
- Token-based authentication
- Centralized access management
- Secure credential storage and rotation
- Improved scalability and connection pooling
- Reduced failover time

[Amazon RDS Proxy](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/rds-proxy.html) provides:
- Connection pooling, typicall for DB cluster that encounters "too many connections" errors, serverless application (e.g. AWS Lambda functions)
- Reduced failover time by up to 66% for Aurora Multi-AZ databases
- Enhanced security with IAM authentication and Secrets Manager
- Automatic scaling of connections

Let's explore how to use these advanced connection features! 💪

## 1. Environment Setup

First, let's set up environment variables for connection management.

In [ ]:
%%bash
echo "Setup connection variables..."

# Allow user to specify Aurora cluster name
AURORA_CLUSTER=""  # Set this to your Aurora cluster name where the data is loaded to, e.g. aurora-sv2-xxx-cluster
AWS_REGION=""  # Set your AWS region where your Aurora cluster resides, e.g. us-east-1

# Check if AWS_REGION is available
if [ -z "$AWS_REGION" ]; then
    echo "❌ AWS_REGION is not set"
    echo "Please set the AWS_REGION variable"
    exit 0
fi

if [ -z "$AURORA_CLUSTER" ]; then
    echo "No specific cluster provided, searching for clusters with cookbook tag..."
    # Find Aurora clusters with CreationSource tag and matching names - get most recent
    AURORA_CLUSTER=$(aws rds describe-db-clusters --region $AWS_REGION --query 'DBClusters[?(contains(DBClusterIdentifier, `aurora-sv2`) || DBClusterIdentifier==`aurora-demo`) && Tags[?Key==`CreationSource` && Value==`aws-database-cookbook-v2025.8`]] | sort_by(@, &ClusterCreateTime) | [-1].DBClusterIdentifier' --output text)
    
    if [ -z "$AURORA_CLUSTER" ]; then
        echo "❌ No Aurora cluster found with cookbook tag"
        echo "Create an Aurora cluster following the steps in Section 2 - Creating Your First Aurora Cluster"
        echo "Available clusters:"
        aws rds describe-db-clusters --region $AWS_REGION --query 'DBClusters[*].{Identifier:DBClusterIdentifier,Engine:Engine,Status:Status}' --output table
        exit 0
    fi
else
    echo "Using specified Aurora cluster: $AURORA_CLUSTER"
fi

echo "Found Aurora cluster: $AURORA_CLUSTER"

# Get managed secret from Aurora cluster
SECRET_ARN=$(aws rds describe-db-clusters --region $AWS_REGION --db-cluster-identifier $AURORA_CLUSTER --query 'DBClusters[0].MasterUserSecret.SecretArn' --output text 2>/dev/null)

if [ "$SECRET_ARN" != "None" ] && [ ! -z "$SECRET_ARN" ]; then
    SECRET_NAME=$SECRET_ARN
    echo "Using managed secret: $SECRET_NAME"
else
    # Fallback for CloudFormation stacks - find by ClusterEndpoint containing 'aurora-sv2'
    CF_STACK_NAME=""
    for stack in $(aws cloudformation list-stacks --region $AWS_REGION --stack-status-filter CREATE_COMPLETE UPDATE_COMPLETE --query 'StackSummaries[*].StackName' --output text); do
        CLUSTER_ENDPOINT=$(aws cloudformation describe-stacks --region $AWS_REGION --stack-name $stack --query "Stacks[0].Outputs[?OutputKey=='ClusterEndpoint'].OutputValue" --output text 2>/dev/null)
        if [ ! -z "$CLUSTER_ENDPOINT" ] && [ "$CLUSTER_ENDPOINT" != "None" ] && [[ "$CLUSTER_ENDPOINT" == *"aurora-sv2"* ]]; then
            CF_STACK_NAME=$stack
            SECRET_NAME=$(aws cloudformation describe-stacks --region $AWS_REGION --stack-name $stack --query "Stacks[0].Outputs[?OutputKey=='SecretArn'].OutputValue" --output text)
            AURORA_CLUSTER=$(echo $CLUSTER_ENDPOINT | cut -d'.' -f1)
            WRITER_ENDPOINT=$CLUSTER_ENDPOINT
            echo "Using CloudFormation secret: $SECRET_NAME"
            echo "Found Aurora cluster from CloudFormation: $AURORA_CLUSTER"
            break
        fi
    done
    
    if [ -z "$CF_STACK_NAME" ]; then
        echo "❌ No managed secret found"
        exit 1
    fi
fi

# Check for existing RDS Proxy
PROXY_NAME=$(aws rds describe-db-proxies --region $AWS_REGION --query "DBProxies[?Tags[?Key=='CreationSource' && Value=='aws-database-cookbook-v2025.8']].DBProxyName" --output text | head -1)
if [ "$PROXY_NAME" != "None" ] && [ -n "$PROXY_NAME" ]; then
    PROXY_ENDPOINT=$(aws rds describe-db-proxies --region $AWS_REGION --db-proxy-name $PROXY_NAME --query 'DBProxies[0].Endpoint' --output text 2>/dev/null || echo "")
    echo "Found existing RDS Proxy: $PROXY_NAME"
fi

# Get Aurora cluster endpoint
WRITER_ENDPOINT=$(aws rds describe-db-clusters --region $AWS_REGION --db-cluster-identifier $AURORA_CLUSTER --query 'DBClusters[0].Endpoint' --output text)

# Save to .proxy_vars
cat > .proxy_vars << EOF
export AURORA_CLUSTER=$AURORA_CLUSTER
export SECRET_NAME=$SECRET_NAME
export PROXY_NAME=$PROXY_NAME
export PROXY_ENDPOINT=$PROXY_ENDPOINT
export WRITER_ENDPOINT=$WRITER_ENDPOINT
export AWS_REGION=$AWS_REGION
export DB_PORT=5432
export DB_NAME=testdb
export DB_USER=app_user
EOF

echo "✅ Environment detected and saved to .proxy_vars:"
echo "Aurora Cluster: $AURORA_CLUSTER"
echo "Secret: $SECRET_NAME"
echo "Proxy: $PROXY_NAME"
echo "Writer Endpoint: $WRITER_ENDPOINT"

## 2. Create Database User and Secrets

### 2.1 Create app_user in Database

First, we'll create the app_user in the database with a generated password stored in Secrets Manager.

In [ ]:
%%bash
source .proxy_vars
echo "Connecting to Aurora cluster..."

# Get master user credentials from Secrets Manager
SECRET_ARN=$(aws secretsmanager describe-secret --secret-id $SECRET_NAME --query 'ARN' --output text)
DB_PASSWORD=$(aws secretsmanager get-secret-value --secret-id $SECRET_ARN --query 'SecretString' --output text | jq -r '.password')

# Generate unique secret name and password for app_user
SECRET_SUFFIX=$(openssl rand -hex 4)
APP_USER_SECRET_NAME="app-user-credentials-$SECRET_SUFFIX"
APP_USER_PASSWORD=$(openssl rand -base64 12 | tr -d "=+/" | cut -c1-16)

# Create secret for app_user
APP_USER_SECRET_ARN=$(aws secretsmanager create-secret \
    --name $APP_USER_SECRET_NAME \
    --description "Credentials for app_user database user" \
    --secret-string "{\"username\":\"app_user\",\"password\":\"$APP_USER_PASSWORD\"}" \
    --query 'ARN' --output text)

# Create app_user in the database
psql "host=$WRITER_ENDPOINT port=5432 dbname=mylab user=masteruser password=$DB_PASSWORD sslmode=require" << EOF
CREATE USER app_user WITH LOGIN PASSWORD '$APP_USER_PASSWORD';
ALTER USER app_user WITH PASSWORD '$APP_USER_PASSWORD';
CREATE DATABASE testdb;
GRANT ALL PRIVILEGES ON DATABASE testdb TO app_user;
\c testdb
CREATE TABLE IF NOT EXISTS test_table (
    id serial PRIMARY KEY,
    name VARCHAR(100),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO app_user;
GRANT ALL PRIVILEGES ON ALL SEQUENCES IN SCHEMA public TO app_user;
INSERT INTO test_table (name) VALUES ('Test User 1'), ('Test User 2');
EOF

echo "export APP_USER_SECRET_ARN=$APP_USER_SECRET_ARN" >> .proxy_vars
echo "User app_user secret: $APP_USER_SECRET_ARN"
echo "✅ app_user created successfully"

### 2.2 Create RDS Proxy with IAM Role

To set up IAM authentication for RDS Proxy in Amazon RDS, create and configure an IAM policy that grants the necessary permissions. RDS Proxy uses AWS Secrets Manager to manage database credentials securely, which allows applications to authenticate through the proxy without directly handling credentials.

In [ ]:
%%bash
echo "Creating IAM roles for RDS proxy using IAM authentication..."

# Get account ID and use AWS_REGION from environment
source .proxy_vars
ACCOUNT_ID=$(aws sts get-caller-identity --query 'Account' --output text)

# Generate unique suffix for role and policy names
RANDOM_SUFFIX=$(openssl rand -hex 4)
ROLE_NAME="rds-proxy-role-$RANDOM_SUFFIX"
POLICY_NAME="app-user-secrets-policy-$RANDOM_SUFFIX"

# Create IAM role for RDS Proxy
echo "Creating IAM role for RDS Proxy: $ROLE_NAME"
aws iam create-role \
    --role-name $ROLE_NAME \
    --assume-role-policy-document '{
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "rds.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }'

# Create policy for Secrets Manager access
echo "Creating policy for Secrets Manager access: $POLICY_NAME"
aws iam create-policy \
    --policy-name $POLICY_NAME \
    --policy-document '{
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "secretsmanager:GetSecretValue",
                "Resource": [
                    "arn:aws:secretsmanager:'$AWS_REGION':'$ACCOUNT_ID':secret:*"
                ]
            },
            {
                "Effect": "Allow",
                "Action": "kms:Decrypt",
                "Resource": "arn:aws:kms:'$AWS_REGION':'$ACCOUNT_ID':key/*",
                "Condition": {
                    "StringEquals": {
                        "kms:ViaService": "secretsmanager.'$AWS_REGION'.amazonaws.com"
                    }
                }
            }
        ]
    }'

# Attach policies to role
aws iam attach-role-policy \
    --role-name $ROLE_NAME \
    --policy-arn arn:aws:iam::$ACCOUNT_ID:policy/$POLICY_NAME

# Save role name for later use
echo "export RDS_PROXY_ROLE_NAME=$ROLE_NAME" >> .proxy_vars
echo "export RDS_PROXY_POLICY_NAME=$POLICY_NAME" >> .proxy_vars

echo "✅ IAM role $ROLE_NAME created with necessary permissions"

Create the RDS Proxy using the IAM role created:

In [ ]:
%%bash
source .proxy_vars

if [ "$PROXY_NAME" == "None" ] || [ -z "$PROXY_NAME" ]; then
    echo "Creating RDS Proxy for cluster $AURORA_CLUSTER..."
    
    # Get subnet and security group info
    SUBNET_GROUP=$(aws rds describe-db-clusters --db-cluster-identifier $AURORA_CLUSTER --query 'DBClusters[0].DBSubnetGroup' --output text)
    SUBNET_IDS=$(aws rds describe-db-subnet-groups --db-subnet-group-name $SUBNET_GROUP --query 'DBSubnetGroups[0].Subnets[].SubnetIdentifier' --output text)
    SECURITY_GROUP=$(aws rds describe-db-clusters --db-cluster-identifier $AURORA_CLUSTER --query 'DBClusters[0].VpcSecurityGroups[0].VpcSecurityGroupId' --output text)
    
    # Get role ARN from saved variables
    ACCOUNT_ID=$(aws sts get-caller-identity --query 'Account' --output text)
    ROLE_ARN="arn:aws:iam::$ACCOUNT_ID:role/$RDS_PROXY_ROLE_NAME"
    
    # Create RDS Proxy
    PROXY_NAME="${AURORA_CLUSTER}-proxy"
    aws rds create-db-proxy \
        --db-proxy-name $PROXY_NAME \
        --engine-family POSTGRESQL \
        --role-arn $ROLE_ARN \
        --vpc-subnet-ids $SUBNET_IDS \
        --vpc-security-group-ids $SECURITY_GROUP \
        --auth AuthScheme=SECRETS,SecretArn=$APP_USER_SECRET_ARN,IAMAuth=REQUIRED \
        --require-tls \
        --idle-client-timeout 1800
    
    # Wait for proxy to be available
    until aws rds describe-db-proxies --db-proxy-name $PROXY_NAME --query 'DBProxies[0].Status' --output text | grep -q "available"; do
        echo "Waiting for RDS proxy to be available..."
        sleep 30
    done
    
    # Create target group
    aws rds register-db-proxy-targets \
        --db-proxy-name $PROXY_NAME \
        --target-group-name default \
        --db-cluster-identifiers $AURORA_CLUSTER
    
    # Get proxy endpoint and update .proxy_vars
    PROXY_ENDPOINT=$(aws rds describe-db-proxies --db-proxy-name $PROXY_NAME --query 'DBProxies[0].Endpoint' --output text)
    sed -i '' '/export PROXY_NAME=/d' .proxy_vars
    sed -i '' '/export PROXY_ENDPOINT=/d' .proxy_vars
    echo "export PROXY_NAME=$PROXY_NAME" >> .proxy_vars
    echo "export PROXY_ENDPOINT=$PROXY_ENDPOINT" >> .proxy_vars
    
    echo "✅ RDS Proxy created: $PROXY_NAME"
    echo "Proxy Endpoint: $PROXY_ENDPOINT"
else
    echo "✅ RDS Proxy already exists: $PROXY_NAME"
fi

## 3. Connect using RDS Proxy with IAM Authentication

Now we'll test the connection through RDS Proxy using IAM authentication. The following code demonstrates how to connect to Aurora PostgreSQL through RDS Proxy using IAM tokens, retrieve app_user credentials from Secrets Manager, and perform database operations with proper error handling and connection management.

In [ ]:
import boto3
import json
import psycopg
import os

# Load variables from .proxy_vars
with open('.proxy_vars', 'r') as f:
    for line in f:
        if line.startswith('export '):
            key, value = line.strip()[7:].split('=', 1)
            os.environ[key] = value

def get_auth_token(host, port, username):
    client = boto3.client('rds')
    return client.generate_db_auth_token(
        DBHostname=host, Port=int(port), DBUsername=username)

def connect_with_iam_auth():
    try:
        proxy_endpoint = os.environ['PROXY_ENDPOINT']
        if not proxy_endpoint:
            raise Exception('No RDS Proxy endpoint found')
        
        auth_token = get_auth_token(proxy_endpoint, os.environ['DB_PORT'], os.environ['DB_USER'])
        
        conn = psycopg.connect(
            host=proxy_endpoint,
            port=int(os.environ['DB_PORT']),
            dbname=os.environ['DB_NAME'],
            user=os.environ['DB_USER'],
            password=auth_token,
            sslmode='require')
        
        print("✅ Connected successfully through RDS Proxy with IAM authentication!")
        return conn
    except Exception as e:
        print(f"❌ Connection failed: {str(e)}")
        return None

# Test connection
conn = connect_with_iam_auth()
if conn:
    try:
        cur = conn.cursor()
        cur.execute("SELECT current_timestamp, current_user;")
        result = cur.fetchone()
        print(f"Current time: {result[0]}, Current user: {result[1]}")
        
        cur.execute("INSERT INTO test_table (name) VALUES (%s) RETURNING id;", ('Connected via RDS Proxy',))
        new_id = cur.fetchone()[0]
        print(f"Inserted new record with ID: {new_id}")
        
        cur.execute("SELECT id, name, created_at FROM test_table ORDER BY id DESC LIMIT 3;")
        records = cur.fetchall()
        print("Latest records:")
        for record in records:
            print(f"ID: {record[0]}, Name: {record[1]}, Created: {record[2]}")
        
        conn.commit()
    except Exception as e:
        print(f"Database operation failed: {str(e)}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()
        print("✅ Connection closed successfully")

## 4. Monitoring and Optimization

Let's set up monitoring for our connection management.

### Prerequisites
- AWS CLI configured
- Required IAM permissions

### Monitor RDS Proxy metrics
The follow examples demonstrate how to monitor RDS Proxy using CloudWatch metrics and AWS CLI.

First, let's set up our environment variables:

In [ ]:
%%bash
source .proxy_vars

export END_TIME=$(date -u +"%Y-%m-%dT%H:%M:%SZ")
export START_TIME=$(date -u -d '1 hour ago' +"%Y-%m-%dT%H:%M:%SZ")

# Save time variables to .proxy_vars
echo "export END_TIME=$END_TIME" >> .proxy_vars
echo "export START_TIME=$START_TIME" >> .proxy_vars

echo "Using RDS Proxy: $PROXY_NAME"

### 4.1. Key Metrics to Monitor

#### A. Connection Pool Utilization

- **DatabaseConnections**: The current number of database connections. This metric is reported every minute. The most useful statistic for this metric is Sum. 
- **MaxDatabaseConnectionsAllowed**: The maximum number of database connections allowed. This metric is reported every minute. The most useful statistic for this metric is Sum. 


In [ ]:
%%bash
source .proxy_vars

echo "Fetching Connection Pool Metrics..."

# Get DatabaseConnections and MaxDatabaseConnectionsAllowed
aws cloudwatch get-metric-data \
    --metric-data-queries '[
        {
            "Id": "connections",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "DatabaseConnections",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            }
        },
        {
            "Id": "maxConnections",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "MaxDatabaseConnectionsAllowed",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Maximum"
            }
        }
    ]' \
    --start-time $START_TIME \
    --end-time $END_TIME \
    --output json > /tmp/metrics.json

echo "
📊 Connection Pool Metrics:"
if [ -s /tmp/metrics.json ] && jq -e '.MetricDataResults' /tmp/metrics.json > /dev/null 2>&1; then
    jq -r '.MetricDataResults[] | "
🔗 " + .Label + " (" + (.Unit // "Count") + "):" + if (.Values | length) > 0 then ("
" + ([.Timestamps, .Values] | transpose | map("  " + .[0] + " | " + (.[1] | tostring)) | join("
"))) else "
  No data available" end' /tmp/metrics.json
else
    echo "⚠️  No metrics data available - this is normal for new proxies"
fi
rm -f /tmp/metrics.json

#### B. Active Connection Analysis

- **DatabaseConnectionsCurrentlyBorrowed**: The current number of database connections in the borrow state. This metric is reported every minute. The most useful statistic for this metric is Sum. 
- **DatabaseConnectionsCurrentlyInTransaction**: The current number of database connections in a transaction. This metric is reported every minute. The most useful statistic for this metric is Sum. 

In [ ]:
%%bash
source .proxy_vars

echo "Fetching Active Connection Metrics..."

aws cloudwatch get-metric-data \
    --metric-data-queries '[
        {
            "Id": "borrowed",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "DatabaseConnectionsCurrentlyBorrowed",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            }
        },
        {
            "Id": "transactions",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "DatabaseConnectionsCurrentlyInTransaction",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            }
        },
        {
            "Id": "clients",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "ClientConnections",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            }
        }
    ]' \
    --start-time $START_TIME \
    --end-time $END_TIME \
    --output json > /tmp/metrics2.json

echo "
📊 Active Connection Analysis:"
if [ -s /tmp/metrics2.json ] && jq -e '.MetricDataResults' /tmp/metrics2.json > /dev/null 2>&1; then
    jq -r '.MetricDataResults[] | "
🔗 " + .Label + " (" + (.Unit // "Count") + "):" + if (.Values | length) > 0 then ("
" + ([.Timestamps, .Values] | transpose | map("  " + .[0] + " | " + (.[1] | tostring)) | join("
"))) else "
  No data available" end' /tmp/metrics2.json
else
    echo "⚠️  No metrics data available - this is normal for new proxies"
fi
rm -f /tmp/metrics2.json

#### C. Performance Metrics

- **QueryResponseLatency**: The time in **microseconds** between getting a query request and the proxy responding to it. The most useful statistic for this metric is Average. 
- **QueryDatabaseResponseLatency**: The time in **microseconds** that the database took to respond to the query. The most useful statistic for this metric is Average. 

Key points to understand:

- QueryResponseLatency will typically be higher than QueryDatabaseResponseLatency because it includes additional processing time within the proxy.
- The difference between these metrics can help identify if there are any performance bottlenecks in the proxy itself or in the communication between the proxy and the database.
- These metrics are useful for monitoring the performance of your RDS Proxy setup and can help in troubleshooting and optimizing your database access patterns. 

In [ ]:
%%bash
source .proxy_vars

echo "Fetching Performance Metrics..."

aws cloudwatch get-metric-data \
    --metric-data-queries '[
        {
            "Id": "queryLatency",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "QueryResponseLatency",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Average"
            }
        },
        {
            "Id": "dbLatency",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "QueryDatabaseResponseLatency",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Average"
            }
        }
    ]' \
    --start-time $START_TIME \
    --end-time $END_TIME \
    --output json > /tmp/metrics3.json

echo "
📊 Performance Metrics:"
if [ -s /tmp/metrics3.json ] && jq -e '.MetricDataResults' /tmp/metrics3.json > /dev/null 2>&1; then
    jq -r '.MetricDataResults[] | "
🔗 " + .Label + " (" + (.Unit // "Microseconds") + "):" + if (.Values | length) > 0 then ("
" + ([.Timestamps, .Values] | transpose | map("  " + .[0] + " | " + (.[1] | tostring)) | join("
"))) else "
  No data available" end' /tmp/metrics3.json
else
    echo "⚠️  No metrics data available - this is normal for new proxies"
fi
rm -f /tmp/metrics3.json

### 4.2. Important Monitoring Considerations

1. **Connection Pool Management**
- Monitor utilization ratio trends for capacity planning
- Account for RDS Proxy fleet size when interpreting metrics
- Watch for periods of high utilization that might indicate need for scaling
- Set different thresholds for peak vs. off-peak hours
- High idle connections percentage might indicate:
  * Inefficient connection pool usage
  * Over-provisioned pool size
  * Connection leaks
- Consider adjusting MaxIdleConnectionsPercent if consistently high

2. **Connection Efficiency**
- Higher multiplexing ratio indicates better connection reuse
- Low multiplexing ratio might indicate:
  * Inefficient connection management
  * Transaction pinning
  * Sub-optimal application patterns
- Monitor transaction durations to identify long-running transactions
- Track connection patterns during peak hours

3. **Performance Analysis**
- Compare QueryResponseLatency with QueryDatabaseResponseLatency to isolate issues
- Use multiple statistics (Average, Max, p99) for comprehensive analysis
- Enable and monitor Performance Insights for detailed query analysis
- Monitor across all dimension sets:
  * ProxyName
  * ProxyName, EndpointName
  * ProxyName, TargetGroup, Target
  * ProxyName, TargetGroup, TargetRole

### 4.3. Creating CloudWatch Alarms

First, let's review the current metrics before creating alarms:

In [ ]:
%%bash
source .proxy_vars

# Display alarm metrics
aws cloudwatch get-metric-data \
    --metric-data-queries '[
        {
            "Id": "e1",
            "Expression": "m1/m2*100",
            "Label": "Connection Pool Utilization %"
        },
        {
            "Id": "m1",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "DatabaseConnections",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            },
            "ReturnData": false
        },
        {
            "Id": "m2",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "MaxDatabaseConnectionsAllowed",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Maximum"
            },
            "ReturnData": false
        },
        {
            "Id": "e2",
            "Expression": "m3/m4",
            "Label": "Multiplexing Ratio"
        },
        {
            "Id": "m3",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "ClientConnections",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            },
            "ReturnData": false
        },
        {
            "Id": "m4",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "DatabaseConnections",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            },
            "ReturnData": false
        },
        {
            "Id": "queryLatency",
            "Label": "QueryResponseLatency",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "QueryResponseLatency",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Average"
            }
        }
    ]' \
    --start-time $START_TIME \
    --end-time $END_TIME \
    --output json > /tmp/alarm_metrics.json

echo "
📊 Alarm Metrics Review:"
if [ -s /tmp/alarm_metrics.json ] && jq -e '.MetricDataResults' /tmp/alarm_metrics.json > /dev/null 2>&1; then
    # Process Connection Pool Utilization with rounding to x.xx
    jq -r '.MetricDataResults[] | select(.Label == "Connection Pool Utilization %") | "
🔗 " + .Label + " (Percent):" + if (.Values | length) > 0 then ("
" + ([.Timestamps, .Values] | transpose | map("  " + .[0] + " | " + (.[1] | tonumber | (. * 100 + 0.5 | floor) / 100 | tostring)) | join("
"))) else "
  No data available" end' /tmp/alarm_metrics.json
    
    # Process Multiplexing Ratio
    jq -r '.MetricDataResults[] | select(.Label == "Multiplexing Ratio") | "
🔗 " + .Label + " (Ratio):" + if (.Values | length) > 0 then ("
" + ([.Timestamps, .Values] | transpose | map("  " + .[0] + " | " + (.[1] | tostring)) | join("
"))) else "
  No data available" end' /tmp/alarm_metrics.json
    
    # Process QueryResponseLatency
    jq -r '.MetricDataResults[] | select(.Label == "QueryResponseLatency") | "
🔗 " + .Label + " (Microseconds):" + if (.Values | length) > 0 then ("
" + ([.Timestamps, .Values] | transpose | map("  " + .[0] + " | " + (.[1] | tostring)) | join("
"))) else "
  No data available" end' /tmp/alarm_metrics.json
else
    echo "⚠️  No metrics data available - this is normal for new proxies"
fi
rm -f /tmp/alarm_metrics.json

echo "
📊 Alarm Thresholds:"
echo "1. Connection Pool Utilization: >80%"
echo "2. Multiplexing Ratio: <2"
echo "3. Query Latency: >1000ms"

If you want to set up automated monitoring, you can create CloudWatch alarms for the key metrics:

In [ ]:
%%bash
source .proxy_vars

echo "Do you want to create CloudWatch alarms? (y/n)"
echo "Set CREATE_ALARMS=y to create alarms, or leave empty to skip"
CREATE_ALARMS=""  # Set to 'y' to create alarms

if [ "$CREATE_ALARMS" = "y" ] || [ "$CREATE_ALARMS" = "Y" ]; then
    echo "Creating CloudWatch alarms..."
    
    # 1. High Connection Pool Utilization Alarm
    aws cloudwatch put-metric-alarm \
        --alarm-name connection-pool-utilization-high \
        --comparison-operator GreaterThanThreshold \
        --evaluation-periods 3 \
        --metrics '[{
            "Id": "e1",
            "Expression": "m1/m2*100",
            "Label": "Connection Pool Utilization %"
        },{
            "Id": "m1",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "DatabaseConnections",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            },
            "ReturnData": false
        },{
            "Id": "m2",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "MaxDatabaseConnectionsAllowed",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Maximum"
            },
            "ReturnData": false
        }]' \
        --threshold 80 \
        --alarm-description "Connection pool utilization exceeded 80%"
    
    # 2. Low Multiplexing Ratio Alarm
    aws cloudwatch put-metric-alarm \
        --alarm-name low-multiplexing-ratio \
        --comparison-operator LessThanThreshold \
        --evaluation-periods 3 \
        --metrics '[{
            "Id": "e1",
            "Expression": "m1/m2",
            "Label": "Multiplexing Ratio"
        },{
            "Id": "m1",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "ClientConnections",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            },
            "ReturnData": false
        },{
            "Id": "m2",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "DatabaseConnections",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Sum"
            },
            "ReturnData": false
        }]' \
        --threshold 2 \
        --alarm-description "Low connection multiplexing efficiency"
    
    # 3. High Query Latency Alarm
    aws cloudwatch put-metric-alarm \
        --alarm-name high-query-latency \
        --comparison-operator GreaterThanThreshold \
        --evaluation-periods 3 \
        --metrics '[{
            "Id": "m1",
            "MetricStat": {
                "Metric": {
                    "Namespace": "AWS/RDS",
                    "MetricName": "QueryResponseLatency",
                    "Dimensions": [{"Name": "ProxyName", "Value": "'$PROXY_NAME'"}]
                },
                "Period": 300,
                "Stat": "Average"
            }
        }]' \
        --threshold 1000 \
        --alarm-description "Query latency exceeded 1 second"
    
    echo "✅ CloudWatch alarms created successfully"
else
    echo "Skipping alarm creation"
fi

## Cleanup

Clean up temporary files created during this workshop:

In [ ]:
%%bash
# Cleanup temporary files
rm -f .proxy_vars
echo "✅ Temporary files cleaned up"

## Next Steps

Congratulations! You've completed the Aurora PostgreSQL connection series. Consider these next steps:

1. **Production Readiness**
   - Implement automated secret rotation
   - Set up comprehensive monitoring
   - Create connection retry logic
   - Test failover scenarios

2. **Security Hardening**
   - Database activity audits via [Database Activity Streams](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/DBActivityStreams.html)
   - Threat detection and vulnerability management via [AWS Security Hub](https://aws.amazon.com/security-hub/)

3. **Advanced Features**
   - Explore [Amazon Aurora Global Database](https://aws.amazon.com/rds/aurora/global-database/)
   - Implement [cross-region disaster recovery](https://d1.awsstatic.com/Amazon%20Aurora%20High%20Availability%20and%20Disaster%20Recovery%20Features%20for%20Global%20Resilience%20Whitepaper.pdf)
   - Try [Aurora Blue/Green Deployment](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/blue-green-deployments.html) for database maintenance

## Additional Resources 📚

### Security & Authentication
- [IAM Authentication Guide](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/UsingWithRDS.IAMDBAuth.html)
- [Secrets Manager User Guide](https://docs.aws.amazon.com/secretsmanager/latest/userguide/)
- [Aurora Security Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.Security.html)

### Connection Management
- [RDS Proxy Documentation](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/rds-proxy.html)
- [Connection Pooling Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/rds-proxy-planning.html)
- [RDS Proxy Monitoring](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/rds-proxy.monitoring.html)
- [Set up highly available PgBouncer and HAProxy with Amazon Aurora PostgreSQL readers](https://aws.amazon.com/blogs/database/set-up-highly-available-pgbouncer-and-haproxy-with-amazon-aurora-postgresql-readers/)
- [PgCat: support for sharding, load balancing, failover and mirroring](https://github.com/postgresml/pgcat)

### Monitoring & Optimization
- [CloudWatch Metrics for RDS](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/monitoring-cloudwatch.html)
- [Performance Insights](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.html)
- [Aurora Monitoring Guide](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/MonitoringAurora.html)

Happy building! 🎉